In [64]:
import bpy
import databpy as db
import polars as pl

obj = bpy.data.objects["Cube"]
evaluated_obj = db.evaluate_object(obj)
mesh = evaluated_obj.to_mesh()

# Collect all attribute data
attribute_data = {}
for attr in mesh.attributes:
    if attr.name not in {'sharp_face', 'UVMap'} and not attr.name.startswith('.'):
        a = db.named_attribute(evaluated_obj, attr.name)
        attribute_data[attr.name] = a

# Create polars DataFrame
df = pl.DataFrame(attribute_data)

# Sort columns so "position" is first
if "position" in df.columns:
    column_order = ["position"] + [col for col in df.columns if col != "position"]
    df = df.select(column_order)

print(df)

shape: (8, 4)
┌────────────────────┬───────┬──────────┬─────────────────────────────────┐
│ position           ┆ hello ┆ world!!! ┆ advanced                        │
│ ---                ┆ ---   ┆ ---      ┆ ---                             │
│ array[f64, 3]      ┆ i64   ┆ bool     ┆ array[f64, 4]                   │
╞════════════════════╪═══════╪══════════╪═════════════════════════════════╡
│ [-1.0, -1.0, -1.0] ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… │
│ [-1.0, -1.0, 1.0]  ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… │
│ [-1.0, 1.0, -1.0]  ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… │
│ [-1.0, 1.0, 1.0]   ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… │
│ [1.0, -1.0, -1.0]  ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… │
│ [1.0, -1.0, 1.0]   ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… │
│ [1.0, 1.0, -1.0]   ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… │
│ [1.0, 1.0, 1.0]    ┆ 42    ┆ true     ┆ [0.992774, 0.11234, … 0.016647… 

In [69]:
# Check dtypes and expand array columns
dtypes = df.dtypes
print("Column dtypes:", dtypes)

# Get columns that are arrays and expand them
array_columns = []
expanded_df = df.clone()

for i, dtype in enumerate(dtypes):
    col_name = df.columns[i]
    if str(dtype).startswith('Array'):
        array_columns.append(col_name)
        print(f"Array column found: {col_name} with dtype {dtype}")
        
        # Get the array length from the first non-null value
        first_array = expanded_df.select(pl.col(col_name)).item(0, 0)
        array_length = len(first_array)
        
        # Expand array into separate columns with indexed names
        for j in range(array_length):
            expanded_df = expanded_df.with_columns([
                pl.col(col_name).arr.get(j).alias(f"{col_name}{j+1}")
            ])
        # Drop the original array column
        expanded_df = expanded_df.drop(col_name)

# Reorder columns to place position columns first
position_columns = [col for col in expanded_df.columns if col.startswith('position')]
other_columns = [col for col in expanded_df.columns if not col.startswith('position')]
column_order = position_columns + other_columns
expanded_df = expanded_df.select(column_order)

print("Array columns:", array_columns)
print("DataFrame after expansion:")
print(expanded_df)

Column dtypes: [Array(Float64, shape=(3,)), Int64, Boolean, Array(Float64, shape=(4,))]
Array column found: position with dtype Array(Float64, shape=(3,))
Array column found: advanced with dtype Array(Float64, shape=(4,))
Array columns: ['position', 'advanced']
DataFrame after expansion:
shape: (8, 9)
┌───────────┬───────────┬───────────┬───────┬───┬───────────┬───────────┬───────────┬───────────┐
│ position1 ┆ position2 ┆ position3 ┆ hello ┆ … ┆ advanced1 ┆ advanced2 ┆ advanced3 ┆ advanced4 │
│ ---       ┆ ---       ┆ ---       ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ f64       ┆ f64       ┆ f64       ┆ i64   ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════════╪═══════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ -1.0      ┆ -1.0      ┆ -1.0      ┆ 42    ┆ … ┆ 0.992774  ┆ 0.11234   ┆ 0.038769  ┆ 0.016647  │
│ -1.0      ┆ -1.0      ┆ 1.0       ┆ 42    ┆ … ┆ 0.992774  ┆ 0.11234   ┆ 0.038769  ┆ 0.016647  │
│ -1.0     

In [71]:
import tempfile
import os

# Write to temporary file
temp_dir = tempfile.gettempdir()
temp_file = os.path.join(temp_dir, "mesh_attributes.csv")
expanded_df.write_csv(temp_file)
print(f"CSV written to: {temp_file}")
print(expanded_df)

CSV written to: /var/folders/0n/w5m51rrn71db4cvkds08wdg80000gn/T/mesh_attributes.csv
shape: (8, 9)
┌───────────┬───────────┬───────────┬───────┬───┬───────────┬───────────┬───────────┬───────────┐
│ position1 ┆ position2 ┆ position3 ┆ hello ┆ … ┆ advanced1 ┆ advanced2 ┆ advanced3 ┆ advanced4 │
│ ---       ┆ ---       ┆ ---       ┆ ---   ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ f64       ┆ f64       ┆ f64       ┆ i64   ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════════╪═══════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ -1.0      ┆ -1.0      ┆ -1.0      ┆ 42    ┆ … ┆ 0.992774  ┆ 0.11234   ┆ 0.038769  ┆ 0.016647  │
│ -1.0      ┆ -1.0      ┆ 1.0       ┆ 42    ┆ … ┆ 0.992774  ┆ 0.11234   ┆ 0.038769  ┆ 0.016647  │
│ -1.0      ┆ 1.0       ┆ -1.0      ┆ 42    ┆ … ┆ 0.992774  ┆ 0.11234   ┆ 0.038769  ┆ 0.016647  │
│ -1.0      ┆ 1.0       ┆ 1.0       ┆ 42    ┆ … ┆ 0.992774  ┆ 0.11234   ┆ 0.038769  ┆ 0.016647  │
│ 1.0       ┆ -1.0 

In [25]:
for attr in mesh.attributes:
    print(attr.name)

hello
position
.select_vert
world!!!
.edge_verts
.select_edge
.select_poly
sharp_face
.corner_vert
.corner_edge
UVMap


In [26]:
for attr in mesh.attributes:
    if attr.name not in {'sharp_face', 'UVMap'} and not attr.name.startswith('.'):
        a = db.named_attribute(obj, attr.name)
        print(attr.name + "\n" + str(a))

NamedAttributeError: The selected attribute 'hello' does not exist on the mesh.